In [1]:
import pandas as pd

# Load the provided Excel file
file_path = 'Data/NSWFuel/202108.xlsx'
# Clean the data by skipping the first two rows and resetting the headers
data_cleaned = pd.read_excel(file_path, skiprows=2)

# Drop rows that are completely empty
data_cleaned.dropna(how='all', inplace=True)

# Resetting the index for better handling
data_cleaned.reset_index(drop=True, inplace=True)

# Forward fill the missing values in the relevant columns
data_cleaned[['ServiceStationName', 'Address', 'Suburb', 'Postcode', 'Brand']] = data_cleaned[['ServiceStationName', 'Address', 'Suburb', 'Postcode', 'Brand']].fillna(method='ffill')

# Convert the PriceUpdatedDate to a datetime object for better handling
data_cleaned['PriceUpdatedDate'] = pd.to_datetime(data_cleaned['PriceUpdatedDate'], format='%d/%m/%Y %I:%M:%S %p').dt.date

# Display the cleaned data to confirm the changes
data_cleaned.head()



/Users/junli/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/Users/junli/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (
/var/folders/ht/ryk_q36173dg_qtcy535zzy40000gp/T/ipykernel_84445/1045955137.py:15: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data_cleaned[['ServiceStationName', 'Address', 'Suburb', 'Postcode', 'Brand']] = data_cleaned[['ServiceStationName', 'Address', 'Suburb', 'Postcode', 'Brand']].fillna(method='ffill')


,ServiceStationName,Address,Suburb,Postcode,Brand,FuelCode,PriceUpdatedDate,Price
0,Metro Petroleum Ballina,"323 River Street, BALLINA NSW 2478",BALLINA,2478.0,Metro Fuel,E10,2021-08-01,151.9
1,Metro Petroleum Ballina,"323 River Street, BALLINA NSW 2478",BALLINA,2478.0,Metro Fuel,U91,2021-08-01,154.9
2,Metro Petroleum Ballina,"323 River Street, BALLINA NSW 2478",BALLINA,2478.0,Metro Fuel,P95,2021-08-01,164.9
3,Metro Petroleum Ballina,"323 River Street, BALLINA NSW 2478",BALLINA,2478.0,Metro Fuel,PDL,2021-08-01,149.9
4,Metro Petroleum Ballina,"323 River Street, BALLINA NSW 2478",BALLINA,2478.0,Metro Fuel,P98,2021-08-01,172.9


In [2]:
# Step 1: Extract all unique addresses and create a unique ID for each service station
unique_addresses = data_cleaned[['ServiceStationName', 'Address', 'Suburb', 'Postcode']].drop_duplicates().reset_index(drop=True)
unique_addresses['ServiceStationID'] = unique_addresses.index + 1  # Assign a unique ID starting from 1

# Display the unique addresses to verify
unique_addresses.head()

,ServiceStationName,Address,Suburb,Postcode,ServiceStationID
0,Metro Petroleum Ballina,"323 River Street, BALLINA NSW 2478",BALLINA,2478.0,1
1,Metro Petroleum Narwee,"32 Bryant St, NARWEE NSW 2209",NARWEE,2209.0,2
2,Independent Kanahooka,"90-94 Lakeside Dr, KANAHOOKA NSW 2530",KANAHOOKA,2530.0,3
3,Metro Peakhurst,"114 Boundary Road, PEAKHURST NSW 2210",PEAKHURST,2210.0,4
4,Metro Revesby,"6,Doyle Rd, Revesby NSW 2210",Revesby,2210.0,5


In [3]:
print(len(unique_addresses))

2017


Query the geo-location of the service stations

In [7]:
import time
import tqdm
from geopy.geocoders import Nominatim

# Initialize a list to store the geolocation data
geolocation_data = []

# Function to gently query geolocation information
def get_geolocation_data(geolocator, address):
    try:
        location = geolocator.geocode(address)
        if location:
            return (location.longitude, location.latitude)
    except:
        return (None, None)

# Iterate through the unique addresses and query their geolocation
geolocator = Nominatim(user_agent="fuel_price_geolocator")
rows = tqdm.tqdm(unique_addresses.iterrows())
i = 0
for index, row in rows:
    address = row['Address']
    
    geolocation = get_geolocation_data(geolocator, address)

    rows.desc = f"{i+1} / {len(unique_addresses)}: " + address + f" {geolocation}"
    
    geolocation_data.append({
        "ServiceStationID": row['ServiceStationID'],
        "ServiceStationName": row['ServiceStationName'],
        "Address": row['Address'],
        "Suburb": row['Suburb'],
        "Postcode": row['Postcode'],
        "Longitude": geolocation[0],
        "Latitude": geolocation[1]
    })
    
    # To avoid being banned, wait for 2 seconds between requests 
    # (Nominatim allows 1 per 1 sec, we play safe)
    time.sleep(2)
    i += 1

    # Local cache in case of failing after a long time. 
    if i % 50 == 0:
        # Convert the geolocation data to a DataFrame
        geolocation_df = pd.DataFrame(geolocation_data)
        # Save the geolocation data to a CSV file for future use
        geolocation_df.to_csv('Data/NSWFuel/service_station_geolocation.csv', index=False)




0it [00:00, ?it/s]

16 / 2017: 91 Bent St (Cnr Spring St), Grafton NSW 2460 None: 15it [00:42,  2.83s/it]                       


TypeError: 'NoneType' object is not subscriptable

In [ ]:
# Next step: Given an address, use the local database to check its geo location,
# if not found query and update local database